# Классификация текстов: спам-фильтр для SMS

В этом задании вам предстоит взять открытый датасет с SMS-сообщениями, размеченными на спам ("spam") и не спам ("ham"), построить на нем классификатор текстов на эти два класса, оценить его качество с помощью кросс-валидации, протестировать его работу на отдельных примерах, и посмотреть, что будет происходить с качеством, если менять параметры вашей модели.

Загрузите датасет. Описание датасета можно посмотреть <a href="http://www.dt.fee.unicamp.br/~tiago/smsspamcollection">здесь</a>.

Считайте датасет в Python

In [133]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB

In [112]:
df = pd.read_table("SMSSpamCollection.txt", header=None)
df.columns = ['label', 'text']

In [113]:
df.head()

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [114]:
print(df[df.label=='ham'].text.iloc[0])
print(df[df.label=='spam'].text.iloc[0])

Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's


In [115]:
# change labels to 1 for spam and 0 for ham
df.label = (df.label == 'spam').astype(int)

Используя `sklearn.feature_extraction.text.CountVectorizer` со стандартными настройками, получите из списка текстов матрицу признаков $X$.

In [116]:
vectorizer = CountVectorizer()
text = vectorizer.fit_transform(df.text)

Оцените качество классификации текстов с помощью `LogisticRegression()` с параметрами по умолчанию, используя `sklearn.cross_validation.cross_val_score` и посчитав среднее арифметическое качества на отдельных fold'ах. Установите `random_state=2`. Параметр cv задайте равным 10. В качестве метрики качества используйте f1-меру. Получившееся качество - один из ответов, которые потребуются при сдаче задания. Ответ округлить до 1 знака после запятой.

In [117]:
clf = LogisticRegression(random_state=2)
answer = np.round(np.mean(cross_val_score(clf, text, df.label, scoring='f1', cv=10)), 1)
print(answer)

0.9


In [118]:
def write_answer(answer, answer_number=1):
    with open('answer_{}.txt'.format(answer_number), 'w') as file:
        file.write(str(answer))

In [119]:
write_answer(answer, 1)

 А теперь обучите классификатор на всей выборке и спрогнозируйте с его помощью класс для следующих сообщений:

"FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB"

"FreeMsg: Txt: claim your reward of 3 hours talk time"

"Have you visited the last lecture on physics?"

"Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$"

"Only 99$"

Прогнозы классификатора (0 - не спам, 1 - спам), записанные через пробел, будут ответом в одном из вопросов ниже.



In [120]:
clf_pipeline = Pipeline([("vectorizer", CountVectorizer()),
                         ("classifier", clf)])

clf_pipeline.fit(df.text, df.label)

print(clf_pipeline)

Pipeline(memory=None,
     steps=[('vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
       ...nalty='l2', random_state=2, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])


In [121]:
sms = ["FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB",
       "FreeMsg: Txt: claim your reward of 3 hours talk time",
       "Have you visited the last lecture on physics?",
       "Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$",
       "Only 99$"]

In [122]:
answer = clf_pipeline.predict(sms)

In [123]:
write_answer(" ".join(str(v) for v in answer), 2)

Задайте в `CountVectorizer` параметр `ngram_range=(2,2)`, затем `ngram_range=(3,3)`, затем `ngram_range=(1,3)`. Во всех трех случаях измерьте получившееся в кросс-валидации значение `f1-меры`, округлите до второго знака после точки, и выпишете результаты через пробел в том же порядке. В данном эксперименте мы пробовали добавлять в признаки n-граммы для разных диапазонов n - только биграммы, только триграммы, и, наконец, все вместе - униграммы, биграммы и триграммы. Обратите внимание, что статистики по биграммам и триграммам намного меньше, поэтому классификатор только на них работает хуже. В то же время это не ухудшает результат сколько-нибудь существенно, если добавлять их вместе с униграммами, т.к. за счет регуляризации линейный классификатор не склонен сильно переобучаться на этих признаках.

In [124]:
def text_classifier(vectorizer, classifier):
    return Pipeline(
        [("vectorizer", vectorizer),
         ("classifier", classifier)]
    )

In [125]:
answer = []
for ngram_range in [(2, 2), (3, 3), (1, 3)]:
    answer.append(cross_val_score(text_classifier(CountVectorizer(ngram_range=ngram_range),
                                                  clf), df.text, df.label, scoring='f1', cv=10).mean())

In [126]:
print(answer)

[0.8224220664187133, 0.7250161555467377, 0.9251382558648837]


In [127]:
write_answer(" ".join(str(v) for v in np.round(answer, 2)), 3)

Повторите аналогичный п.7 эксперимент, используя вместо логистической регрессии `MultinomialNB()`. Обратите внимание, насколько сильнее (по сравнению с линейным классификатором) наивный Байес страдает от нехватки статистики по биграммам и триграммам.

По какой-то причине обучение наивного байесовского классификатора через `Pipeline` происходит с ошибкой. Чтобы получить правильный ответ, отдельно посчитайте частоты слов и обучите классификатор. 

In [128]:
clf = MultinomialNB()

In [129]:
labels = df.label

answer = []

for ngram_range in [(2, 2), (3, 3), (1, 3)]:
    vectorizer = CountVectorizer(ngram_range=ngram_range)
    text = vectorizer.fit_transform(df.text)
    answer.append(cross_val_score(clf, text, labels, scoring='f1', cv=10).mean())

In [130]:
print(answer)

[0.6455015177985443, 0.37871948524573595, 0.8884859656061002]


In [131]:
write_answer(" ".join(str(v) for v in np.round(answer, 2)), 4)

Попробуйте использовать в логистической регрессии в качестве признаков Tf\*idf из `TfidfVectorizer` на униграммах. Повысилось или понизилось качество на кросс-валидации по сравнению с `CountVectorizer` на униграммах? (напишите в файле с ответом 1, если повысилось, -1, если понизилось, и 0, если изменилось не более чем на 0.01). Обратите внимание, что результат перехода к tf\*idf не всегда будет таким - если вы наблюдаете какое-то явление на одном датасете, не надо сразу же его обобщать на любые данные.

In [132]:
clf = LogisticRegression(random_state=2)
text = CountVectorizer().fit_transform(df.text)
cross_val_score(clf, text, df.label, scoring='f1', cv=10).mean()

0.9326402983610631

In [134]:
text = TfidfVectorizer().fit_transform(df.text)
cross_val_score(clf, text, df.label, scoring='f1', cv=10).mean()

0.8528599554172456

In [135]:
write_answer(-1, 5)